In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
df = pd.read_csv('/content/drive/MyDrive/파이토치/chap02/data/car_evaluation.csv')

In [49]:
df.head()

,price,maint,doors,persons,lug_capacity,safety,output
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [50]:
# 데이터 범주형 타입으로 변환
c_cols = ['price', 'maint', 'doors', 'persons', 'lug_capacity', 'safety']
for c in c_cols:
  df[c] = df[c].astype('category')
price = df['price'].cat.codes.values
maint = df['maint'].cat.codes.values
doors = df['doors'].cat.codes.values
persons = df['persons'].cat.codes.values
lug_capacity = df['lug_capacity'].cat.codes.values
safety = df['safety'].cat.codes.values

cateforical_data = np.stack([price, maint, doors, persons, lug_capacity, safety], 1)
cateforical_data[:10]

array([[3, 3, 0, 0, 2, 1],
       [3, 3, 0, 0, 2, 2],
       [3, 3, 0, 0, 2, 0],
       [3, 3, 0, 0, 1, 1],
       [3, 3, 0, 0, 1, 2],
       [3, 3, 0, 0, 1, 0],
       [3, 3, 0, 0, 0, 1],
       [3, 3, 0, 0, 0, 2],
       [3, 3, 0, 0, 0, 0],
       [3, 3, 0, 1, 2, 1]], dtype=int8)

In [51]:
# 배열을 텐서로 변환
c_data = torch.tensor(cateforical_data, dtype=torch.int64)

In [52]:
# 레이블로 사용할 컬럼을 텐서로 변환
outputs = pd.get_dummies(df.output)
outputs = outputs.values
outputs = torch.tensor(outputs).flatten()

print(c_data.shape)
print(outputs.shape)

torch.Size([1728, 6])
torch.Size([6912])


In [53]:
# 범주형 칼럼을 N차원으로 변환
c_col_sizes = [len(df[column].cat.categories) for column in c_cols]
c_em_sizes = [(col_size, min(50, col_size+1)//2)for col_size in c_col_sizes]
print(c_em_sizes)

[(4, 2), (4, 2), (4, 2), (3, 2), (3, 2), (3, 2)]


In [54]:
# 데이터 셋 분리
total_records = 1728
test_records = int(total_records * .2)

c_train_data = c_data[:total_records - test_records]
c_test_data = c_data[total_records - test_records:total_records]
train_outputs = outputs[:total_records - test_records]
test_outputs = outputs[total_records - test_records:total_records]

In [55]:
print(len(c_train_data))
print(len(train_outputs))
print(len(c_test_data))
print(len(test_outputs))

1383
1383
345
345


In [56]:
# 모델의 네트워크 생성
class Model(nn.Module):
  def __init__(self, embedding_size, output_size, layers, p=0.4):
    super().__init__()
    self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
    self.embedding_dropout = nn.Dropout(p)

    all_layers = []
    num_c_cols = sum((nf for ni, nf in embedding_size))
    input_size = num_c_cols
    for i in layers:
      all_layers.append(nn.Linear(input_size, i))
      all_layers.append(nn.ReLU(inplace=True))
      all_layers.append(nn.BatchNorm1d(i))
      all_layers.append(nn.Dropout(p))
      input_size = i

    all_layers.append(nn.Linear(layers[-1], output_size))
    self.layers = nn.Sequential(*all_layers)

  def forward(self, x_categorical):
    embeddings = []
    for i, e in enumerate(self.all_embeddings):
      embeddings.append(e(x_categorical[:, i]))
    x = torch.cat(embeddings, 1)
    x = self.embedding_dropout(x)
    x = self.layers(x)
    return x

In [57]:
model = Model(c_em_sizes, 4, [200,100,50], p=0.4)
print(model)

Model(
  (all_embeddings): ModuleList(
    (0-2): 3 x Embedding(4, 2)
    (3-5): 3 x Embedding(3, 2)
  )
  (embedding_dropout): Dropout(p=0.4, inplace=False)
  (layers): Sequential(
    (0): Linear(in_features=12, out_features=200, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=200, out_features=100, bias=True)
    (5): ReLU(inplace=True)
    (6): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.4, inplace=False)
    (8): Linear(in_features=100, out_features=50, bias=True)
    (9): ReLU(inplace=True)
    (10): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.4, inplace=False)
    (12): Linear(in_features=50, out_features=4, bias=True)
  )
)


In [58]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [59]:
# 장치 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 데이터를 장치로 이동
c_train_data = c_train_data.to(device)
c_test_data = c_test_data.to(device)
train_outputs = train_outputs.to(device=device, dtype=torch.int64)
test_outputs = test_outputs.to(device=device, dtype=torch.int64)

In [60]:
epochs = 500
aggregated_losses = []
train_outputs = train_outputs.to(device=device, dtype=torch.int64)

for i in range(epochs):
  i += 1
  y_pred = model(c_train_data).to(device)
  single_loss = loss_function(y_pred, train_outputs)
  aggregated_losses.append(single_loss)

  if i%25 == 1:
    print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

  optimizer.zero_grad()
  single_loss.backward()
  optimizer.step()
print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   1 loss: 1.54510951
epoch:  26 loss: 1.35888004
epoch:  51 loss: 1.27301025
epoch:  76 loss: 1.16882718
epoch: 101 loss: 1.04646075
epoch: 126 loss: 0.90359992
epoch: 151 loss: 0.81012654
epoch: 176 loss: 0.73284477
epoch: 201 loss: 0.69063699
epoch: 226 loss: 0.65673012
epoch: 251 loss: 0.62663990
epoch: 276 loss: 0.61789858
epoch: 301 loss: 0.60623139
epoch: 326 loss: 0.59234017
epoch: 351 loss: 0.58574837
epoch: 376 loss: 0.59324294
epoch: 401 loss: 0.57961053
epoch: 426 loss: 0.58675873
epoch: 451 loss: 0.57616830
epoch: 476 loss: 0.56766814
epoch: 500 loss: 0.5790734291


In [61]:
test_outputs = test_outputs.to(device=device, dtype=torch.int64)
with torch.no_grad():
  y_val = model(c_test_data)
  loss = loss_function(y_val, test_outputs)
print(f'Loss: {loss:.8f}')

Loss: 0.57076377


In [62]:
print(y_val[:5])

tensor([[ 2.9930,  1.7883, -2.9416, -3.1178],
        [ 3.0646,  1.7053, -3.4676, -3.5320],
        [ 2.9776,  2.2839, -3.5526, -3.9254],
        [ 2.1366,  1.2644, -2.1853, -2.5315],
        [ 2.4134,  1.3695, -2.2523, -2.5418]], device='cuda:0')


In [65]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Test outputs와 y_val을 CPU로 이동하고 numpy 배열로 변환
test_outputs_cpu = test_outputs.cpu()
y_val_cpu = y_val.cpu()

# 예측값을 가장 높은 확률의 클래스로 변환
_, y_val_pred = torch.max(y_val_cpu, 1)

# 평가 지표 계산
print(confusion_matrix(test_outputs_cpu, y_val_pred))
print(classification_report(test_outputs_cpu, y_val_pred))
print(accuracy_score(test_outputs_cpu, y_val_pred))

[[257   2]
 [ 85   1]]
              precision    recall  f1-score   support

           0       0.75      0.99      0.86       259
           1       0.33      0.01      0.02        86

    accuracy                           0.75       345
   macro avg       0.54      0.50      0.44       345
weighted avg       0.65      0.75      0.65       345

0.7478260869565218
